# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 12 2022 9:40AM,252798,10,0086237788,ISDIN Corporation,Released
1,Dec 12 2022 9:40AM,252798,10,0086237807,ISDIN Corporation,Released
2,Dec 12 2022 9:40AM,252798,10,0086237809,ISDIN Corporation,Released
3,Dec 12 2022 9:40AM,252798,10,0086237811,ISDIN Corporation,Released
4,Dec 12 2022 9:40AM,252798,10,0086237815,ISDIN Corporation,Released
5,Dec 12 2022 9:40AM,252798,10,0086237814,ISDIN Corporation,Released
6,Dec 12 2022 9:40AM,252798,10,0086237816,ISDIN Corporation,Released
7,Dec 12 2022 9:40AM,252798,10,0086237819,ISDIN Corporation,Released
8,Dec 12 2022 9:40AM,252798,10,0086237824,ISDIN Corporation,Released
9,Dec 12 2022 9:40AM,252798,10,0086237826,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,252795,Released,3
17,252796,Released,35
18,252797,Released,37
19,252798,Released,61
20,252800,Released,55


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
252795,NaN,3.0
252796,NaN,35.0
252797,NaN,37.0
252798,NaN,61.0
252800,NaN,55.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
252771,4.0,3.0
252777,0.0,1.0
252778,0.0,1.0
252779,0.0,1.0
252780,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
252771,4,3
252777,0,1
252778,0,1
252779,0,1
252780,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,252771,4,3
1,252777,0,1
2,252778,0,1
3,252779,0,1
4,252780,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,252771,4,3
1,252777,,1
2,252778,,1
3,252779,,1
4,252780,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 12 2022 9:40AM,252798,10,ISDIN Corporation
61,Dec 12 2022 9:39AM,252800,10,ISDIN Corporation
116,Dec 12 2022 9:37AM,252797,10,ISDIN Corporation
153,Dec 12 2022 9:37AM,252796,10,ISDIN Corporation
188,Dec 12 2022 9:36AM,252792,10,"Methapharm, Inc."
216,Dec 12 2022 9:28AM,252795,10,"CLINUVEL, Inc."
219,Dec 12 2022 9:10AM,252793,10,MedStone Pharma LLC
221,Dec 12 2022 9:06AM,252791,10,"Methapharm, Inc."
226,Dec 12 2022 9:02AM,252789,12,"Uniderm USA, Inc"
227,Dec 12 2022 8:58AM,252788,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 12 2022 9:40AM,252798,10,ISDIN Corporation,,61
1,Dec 12 2022 9:39AM,252800,10,ISDIN Corporation,,55
2,Dec 12 2022 9:37AM,252797,10,ISDIN Corporation,,37
3,Dec 12 2022 9:37AM,252796,10,ISDIN Corporation,,35
4,Dec 12 2022 9:36AM,252792,10,"Methapharm, Inc.",,28
5,Dec 12 2022 9:28AM,252795,10,"CLINUVEL, Inc.",,3
6,Dec 12 2022 9:10AM,252793,10,MedStone Pharma LLC,,2
7,Dec 12 2022 9:06AM,252791,10,"Methapharm, Inc.",,5
8,Dec 12 2022 9:02AM,252789,12,"Uniderm USA, Inc",,1
9,Dec 12 2022 8:58AM,252788,10,Emerginnova,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 9:40AM,252798,10,ISDIN Corporation,61,
1,Dec 12 2022 9:39AM,252800,10,ISDIN Corporation,55,
2,Dec 12 2022 9:37AM,252797,10,ISDIN Corporation,37,
3,Dec 12 2022 9:37AM,252796,10,ISDIN Corporation,35,
4,Dec 12 2022 9:36AM,252792,10,"Methapharm, Inc.",28,
5,Dec 12 2022 9:28AM,252795,10,"CLINUVEL, Inc.",3,
6,Dec 12 2022 9:10AM,252793,10,MedStone Pharma LLC,2,
7,Dec 12 2022 9:06AM,252791,10,"Methapharm, Inc.",5,
8,Dec 12 2022 9:02AM,252789,12,"Uniderm USA, Inc",1,
9,Dec 12 2022 8:58AM,252788,10,Emerginnova,6,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 9:40AM,252798,10,ISDIN Corporation,61,
1,Dec 12 2022 9:39AM,252800,10,ISDIN Corporation,55,
2,Dec 12 2022 9:37AM,252797,10,ISDIN Corporation,37,
3,Dec 12 2022 9:37AM,252796,10,ISDIN Corporation,35,
4,Dec 12 2022 9:36AM,252792,10,"Methapharm, Inc.",28,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 9:40AM,252798,10,ISDIN Corporation,61.0,NaN
1,Dec 12 2022 9:39AM,252800,10,ISDIN Corporation,55.0,NaN
2,Dec 12 2022 9:37AM,252797,10,ISDIN Corporation,37.0,NaN
3,Dec 12 2022 9:37AM,252796,10,ISDIN Corporation,35.0,NaN
4,Dec 12 2022 9:36AM,252792,10,"Methapharm, Inc.",28.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 9:40AM,252798,10,ISDIN Corporation,61.0,0.0
1,Dec 12 2022 9:39AM,252800,10,ISDIN Corporation,55.0,0.0
2,Dec 12 2022 9:37AM,252797,10,ISDIN Corporation,37.0,0.0
3,Dec 12 2022 9:37AM,252796,10,ISDIN Corporation,35.0,0.0
4,Dec 12 2022 9:36AM,252792,10,"Methapharm, Inc.",28.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2527935,240.0,0.0
12,758359,2.0,1.0
19,505555,4.0,4.0
20,758341,3.0,0.0
22,505559,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2527935,240.0,0.0
1,12,758359,2.0,1.0
2,19,505555,4.0,4.0
3,20,758341,3.0,0.0
4,22,505559,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,240.0,0.0
1,12,2.0,1.0
2,19,4.0,4.0
3,20,3.0,0.0
4,22,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,240.0
1,12,Released,2.0
2,19,Released,4.0
3,20,Released,3.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20,22
Status,,,,,
Executing,0.0,1.0,4.0,0.0,1.0
Released,240.0,2.0,4.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20,22
0,Executing,0.0,1.0,4.0,0.0,1.0
1,Released,240.0,2.0,4.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20,22
0,Executing,0.0,1.0,4.0,0.0,1.0
1,Released,240.0,2.0,4.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()